In [12]:
import pandas as pd
import numpy as np

In [5]:
# Load the CSV dataset
df = pd.read_csv("Walmart_Sales.csv")

# Display the first rows to check the structure
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [10]:
# Print dataset information in one cell
print(f"Number of rows and columns: {df.shape}\n")
print("Missing values:\n", df.isnull().sum(), "\n")
print("Dataset info:")
df.info()

Number of rows and columns: (6435, 8)

Missing values:
 Store           0
Date            0
Weekly_Sales    0
Holiday_Flag    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64 

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 402.3+ KB


In [11]:
# Convert "Date" to datetime
df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")

# Extract Year, Month, and Week for better analysis
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["Week"] = df["Date"].dt.isocalendar().week

# Sort data by Date
df = df.sort_values(by="Date")

df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2010,2,5
1287,10,2010-02-05,2193048.75,0,54.34,2.962,126.442065,9.765,2010,2,5
5148,37,2010-02-05,536006.73,0,45.97,2.572,209.852966,8.554,2010,2,5
2288,17,2010-02-05,789036.02,0,23.11,2.666,126.442065,6.548,2010,2,5
4147,30,2010-02-05,465108.52,0,39.05,2.572,210.752605,8.324,2010,2,5


In [13]:
# Check for outliers in Weekly_Sales using percentiles
q1 = df["Weekly_Sales"].quantile(0.25)
q3 = df["Weekly_Sales"].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Count outliers
outliers = df[(df["Weekly_Sales"] < lower_bound) | (df["Weekly_Sales"] > upper_bound)]
print(f"Number of outliers in Weekly_Sales: {outliers.shape[0]}")

# Create a column with average sales per store
df["Avg_Store_Sales"] = df.groupby("Store")["Weekly_Sales"].transform("mean")

# Create a column for % variation in sales per store compared to previous week
df["Sales_Variation"] = df.groupby("Store")["Weekly_Sales"].pct_change() * 100

df.head()

Number of outliers in Weekly_Sales: 34


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Week,Avg_Store_Sales,Sales_Variation
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2010,2,5,1.555264e+06,NaN
1287,10,2010-02-05,2193048.75,0,54.34,2.962,126.442065,9.765,2010,2,5,1.899425e+06,NaN
5148,37,2010-02-05,536006.73,0,45.97,2.572,209.852966,8.554,2010,2,5,5.189003e+05,NaN
2288,17,2010-02-05,789036.02,0,23.11,2.666,126.442065,6.548,2010,2,5,8.935814e+05,NaN
4147,30,2010-02-05,465108.52,0,39.05,2.572,210.752605,8.324,2010,2,5,4.385796e+05,NaN


In [14]:
# Cap values: replace extreme values with upper/lower limits
df["Weekly_Sales"] = np.where(df["Weekly_Sales"] < lower_bound, lower_bound, df["Weekly_Sales"])
df["Weekly_Sales"] = np.where(df["Weekly_Sales"] > upper_bound, upper_bound, df["Weekly_Sales"])

# Check new min and max values
print(f"New min: {df['Weekly_Sales'].min()}, New max: {df['Weekly_Sales'].max()}")


New min: 209986.25, New max: 2720371.4924999997


In [15]:
# Replace NaN in Sales_Variation with 0
df["Sales_Variation"] = df["Sales_Variation"].fillna(0)

# Check if there are still NaN values
print(df.isnull().sum())

Store              0
Date               0
Weekly_Sales       0
Holiday_Flag       0
Temperature        0
Fuel_Price         0
CPI                0
Unemployment       0
Year               0
Month              0
Week               0
Avg_Store_Sales    0
Sales_Variation    0
dtype: int64


In [16]:
df.to_csv("walmart_sales_cleaned.csv", index=False)
